In [1]:
import os
import zipfile
import random
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile


In [2]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2020-08-19 12:09:42--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  81.0MB/s    in 1.0s    

2020-08-19 12:09:44 (81.0 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

last layer output shape:  (None, 7, 7, 768)


In [3]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2020-08-19 12:10:19--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.78.216.154, 2600:1417:76:586::e59, 2600:1417:76:58e::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.78.216.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M  99.4MB/s    in 7.4s    

2020-08-19 12:10:27 (106 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [4]:
try:
    
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

In [5]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if (os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file size! i.e Zero length.')
    
    train_data_length = int(len(dataset) * SPLIT_SIZE)
    test_data_length = int(len(dataset) - train_data_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = shuffled_set[0:train_data_length]
    test_set = shuffled_set[-test_data_length:]
    
    for unitData in train_set:
        temp_train_data = SOURCE + unitData
        final_train_data = TRAINING + unitData
        copyfile(temp_train_data, final_train_data)
    
    for unitData in test_set:
        temp_test_data = SOURCE + unitData
        final_test_data = TESTING + unitData
        copyfile(temp_train_data, final_test_data)



CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

Skipped 666.jpg
Invalid file size! i.e Zero length.
Skipped 11702.jpg
Invalid file size! i.e Zero length.


In [6]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if (logs.get('accuracy')>0.99):
      print("Reaches 99% accuracy. Cancelling training")
      self.model.stop_training = True

callbacks = myCallback()

In [11]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense( 1 , activation = 'sigmoid')(x)

model = Model( pre_trained_model.input , x)


model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
TRAINING_DIR = '/tmp/cats-v-dogs/training'
train_datagen = ImageDataGenerator( rescale = 1/255,
                                  rotation_range = 40,
                                   width_shift_range= 0.2,
                                   height_shift_range= 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest'
                                  
                                  )


train_generator = train_datagen.flow_from_directory(
TRAINING_DIR , batch_size = 20 , class_mode = 'binary' , target_size = (150,150))

VALIDATION_DIR = '/tmp/cats-v-dogs/testing'
validation_datagen = ImageDataGenerator( rescale = 1/255)


validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR , batch_size= 20 , class_mode = 'binary' , target_size=(150,150))

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator ,
                    steps_per_epoch=100, 
                    epochs=100, 
                    verbose=2,validation_steps=50,
                    validation_data=validation_generator , callbacks=[callbacks])

Epoch 1/100
100/100 - 25s - loss: 0.3449 - accuracy: 0.8655 - val_loss: 5.6572e-06 - val_accuracy: 1.0000
Epoch 2/100
100/100 - 24s - loss: 0.2184 - accuracy: 0.9120 - val_loss: 6.4348e-07 - val_accuracy: 1.0000
Epoch 3/100
100/100 - 24s - loss: 0.2366 - accuracy: 0.9170 - val_loss: 1.6283e-07 - val_accuracy: 1.0000
Epoch 4/100
100/100 - 24s - loss: 0.2075 - accuracy: 0.9190 - val_loss: 5.5547e-09 - val_accuracy: 1.0000
Epoch 5/100
100/100 - 24s - loss: 0.2231 - accuracy: 0.9165 - val_loss: 1.0316e-08 - val_accuracy: 1.0000
Epoch 6/100
100/100 - 24s - loss: 0.2218 - accuracy: 0.9220 - val_loss: 1.3082e-08 - val_accuracy: 1.0000
Epoch 7/100
100/100 - 24s - loss: 0.1832 - accuracy: 0.9374 - val_loss: 1.9001e-09 - val_accuracy: 1.0000
Epoch 8/100
100/100 - 23s - loss: 0.2029 - accuracy: 0.9270 - val_loss: 8.8861e-10 - val_accuracy: 1.0000
Epoch 9/100
100/100 - 24s - loss: 0.2122 - accuracy: 0.9220 - val_loss: 8.1163e-11 - val_accuracy: 1.0000
Epoch 10/100
100/100 - 23s - loss: 0.2058 - ac

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

100/100 - 23s - loss: 0.2134 - accuracy: 0.9290 - val_loss: 4.7906e-11 - val_accuracy: 1.0000
Epoch 13/100
100/100 - 24s - loss: 0.1908 - accuracy: 0.9289 - val_loss: 1.1717e-10 - val_accuracy: 1.0000
Epoch 14/100
100/100 - 23s - loss: 0.1784 - accuracy: 0.9405 - val_loss: 8.6074e-10 - val_accuracy: 1.0000
Epoch 15/100
100/100 - 24s - loss: 0.1881 - accuracy: 0.9280 - val_loss: 3.9292e-11 - val_accuracy: 1.0000
Epoch 16/100
100/100 - 24s - loss: 0.1778 - accuracy: 0.9405 - val_loss: 7.6988e-09 - val_accuracy: 1.0000
Epoch 17/100
100/100 - 24s - loss: 0.1938 - accuracy: 0.9370 - val_loss: 1.4923e-09 - val_accuracy: 1.0000
Epoch 18/100
100/100 - 24s - loss: 0.1740 - accuracy: 0.9425 - val_loss: 8.5468e-11 - val_accuracy: 1.0000
Epoch 19/100
100/100 - 24s - loss: 0.1910 - accuracy: 0.9345 - val_loss: 2.7923e-10 - val_accuracy: 1.0000
Epoch 20/100
100/100 - 24s - loss: 0.1733 - accuracy: 0.9340 - val_loss: 2.9632e-11 - val_accuracy: 1.0000
Epoch 21/100
100/100 - 24s - loss: 0.1661 - accura

In [10]:

%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt


acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) 


plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()


plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')



NameError: ignored